In [1]:
import pandas as pd
import numpy as np
import random
from rdkit import Chem
from snn_model import get_loss_fn
import torch
from torch.utils.data import TensorDataset, DataLoader
from snntorch import spikegen, surrogate
import matplotlib.pyplot as plt
from utils import load_dataset_df, smile_to_fp, data_splitter, get_spiking_net, make_filename
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, precision_score
from csnn_model import CSNNet


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\knsve\Desktop\MEI\Tese\torch\snn_venv\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-geometri

In [2]:
files = ['tox21.csv','sider.csv', 'BBBP.csv']
dt_file = files[1]
dirname = dt_file.removesuffix('.csv')

df, targets = load_dataset_df(filename=dt_file)

for t in targets:
    df_temp = df[[t, 'smiles']].dropna()
    class_counts = df[t].count()
    class_sum = df[t].sum()
    print(t, class_counts, round(class_sum/class_counts, 2)) 

Hepatobiliary disorders 1427 0.52
Metabolism and nutrition disorders 1427 0.7
Product issues 1427 0.02
Eye disorders 1427 0.61
Investigations 1427 0.81
Musculoskeletal and connective tissue disorders 1427 0.7
Gastrointestinal disorders 1427 0.91
Social circumstances 1427 0.18
Immune system disorders 1427 0.72
Reproductive system and breast disorders 1427 0.51
Neoplasms benign, malignant and unspecified (incl cysts and polyps) 1427 0.26
General disorders and administration site conditions 1427 0.91
Endocrine disorders 1427 0.23
Surgical and medical procedures 1427 0.15
Vascular disorders 1427 0.78
Blood and lymphatic system disorders 1427 0.62
Skin and subcutaneous tissue disorders 1427 0.92
Congenital, familial and genetic disorders 1427 0.18
Infections and infestations 1427 0.7
Respiratory, thoracic and mediastinal disorders 1427 0.74
Psychiatric disorders 1427 0.71
Renal and urinary disorders 1427 0.64
Pregnancy, puerperium and perinatal conditions 1427 0.09
Ear and labyrinth disorde